In [4]:
import graphlab
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

people = graphlab.SFrame('people_wiki.gl/')

In [39]:
#to explore the data set
obama = people[people['name'] == 'Barack Obama']

#to get the word counts for an article
obama['word_count'] = graphlab.text_analytics.count_words(obama['text'])

#stack-takes one column and stacks on top of other
obama_word_count_table = obama[['word_count']].stack('word_count', new_column_name = ['word', 'count']).sort('count', ascending = False)

#to create a column and count the words
people['word_count'] = graphlab.text_analytics.count_words(people['text'])

#to get unimportant words out- compute TF-IDF for the corpus
tfidf = graphlab.text_analytics.tf_idf(people['word_count'])

if graphlab.version <= '1.6.1':
    tfidf = tfidf['docs']

people['tfidf'] = tfidf
    
obama = people[people['name'] == 'Barack Obama']
obama[['tfidf']].stack('tfidf', new_column_name = ['word', 'tfidf']).sort('tfidf', ascending = False)

#to compute distances between people
# clinton = people[people['name'] == 'Bill Clinton']
# beckham = people[people['name'] == 'David Beckham']

#to compute who is closer
# print "Obama vs Clinton", graphlab.distances.cosine(obama['tfidf'][0],clinton['tfidf'][0])
# print "Obama vs Beckham", graphlab.distances.cosine(obama['tfidf'][0],beckham['tfidf'][0])

#computing using the nearest neighbour model for document retrieval
knn_model = graphlab.nearest_neighbors.create(people, features = ['tfidf'], label = 'name')

#who is closest to Obama
knn_model.query(obama)

Starting brute force nearest neighbors model training.

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 42.085ms     |

| Done         |         | 100         | 1.02s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Barack Obama,0.0,1
0,Joe Biden,0.794117647059,2
0,Joe Lieberman,0.794685990338,3
0,Kelly Ayotte,0.811989100817,4
0,Bill Clinton,0.813852813853,5


In [48]:
#question 1
Elton = people[people['name'] == 'Elton John']

#word_count
Elton['word_count'] = graphlab.text_analytics.count_words(Elton['text'])
print Elton[['word_count']].stack('word_count', new_column_name = ['word', 'count']).sort('count', ascending= False)

#tfidf_count
Elton = people[people['name'] == 'Elton John']
Elton['word_count'] = graphlab.text_analytics.count_words(Elton['text'])
tfidf = graphlab.text_analytics.tf_idf(people['word_count'])

if graphlab.version <= '1.6.1':
    tfidf = tfidf['docs']
tfidf

people['tfidf'] = tfidf
print Elton[['tfidf']].stack('tfidf', new_column_name = ['word', 'tfidf']).sort('tfidf', ascending= False)

+-------+-------+
|  word | count |
+-------+-------+
|  the  |   27  |
|   in  |   18  |
|  and  |   15  |
|   of  |   13  |
|   a   |   10  |
|  has  |   9   |
|  john |   7   |
|   he  |   7   |
|   on  |   6   |
| award |   5   |
+-------+-------+
[255 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
+---------------+---------------+
|      word     |     tfidf     |
+---------------+---------------+
|    furnish    |  18.38947184  |
|     elton     |  17.48232027  |
|   billboard   | 17.3036809575 |
|      john     | 13.9393127924 |
|  songwriters  |  11.250406447 |
| tonightcandle | 10.9864953892 |
|  overallelton | 10.9864953892 |
|    19702000   | 10.2933482087 |
|   fivedecade  | 10.2933482087 |
|      aids     |  10.262846934 |
+---------------+---------------+
[255 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to

In [51]:
#question 2

#to compute distances between people
Victoria  = people[people['name'] == 'Victoria Beckham']
Paul = people[people['name'] == 'Paul McCartney']

#to compute who is closer
print "Elton vs Victoria ", graphlab.distances.cosine(Elton['tfidf'][0],Victoria ['tfidf'][0])
print "Elton vs Paul ", graphlab.distances.cosine(Elton['tfidf'][0],Paul ['tfidf'][0])

 Elton vs Victoria  0.956700637666
Elton vs Paul  0.825031002922


In [56]:
#question 3

#question3a
knn_model = graphlab.nearest_neighbors.create(people,features = ['word_count'], distance='cosine', label = 'name')
print "-----------3a-------", knn_model.query(Elton)
print "-----------3c-------", knn_model.query(Victoria)

#question3b
knn_model = graphlab.nearest_neighbors.create(people,features = ['tfidf'], distance='cosine', label = 'name')
print "-----------3b-------", knn_model.query(Elton)
print "-----------3d-------", knn_model.query(Victoria)

Starting brute force nearest neighbors model training.

-----------3a-------

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

...

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 24.35ms      |

| Done         |         | 100         | 955.123ms    |

+--------------+---------+-------------+--------------+

+-------------+---------------------+-------------------+------+
| query_label |   reference_label   |      distance     | rank |
+-------------+---------------------+-------------------+------+
|      0      |   Victoria Beckham  | 1.11022302463e-16 |  1   |
|      0      |    David Beckham    |   0.548169610263  |  2   |
|      0      | Stephen Dow Beckham |   0.784986706828  |  3   |
|      0      |        Mel B        |   0.809585523409  |  4   |
|      0      |    Caroline Rush    |   0.819826422919  |  5   |
+-------------+---------------------+-------------------+------+
[5 rows x 4 columns]

